# Аугментация

In [308]:
import albumentations as alb
import cv2
import os

In [309]:
def gen_aug(index, sub_index):
    img = cv2.imread(f'input\img\image{index}.jpg')
    mask = cv2.imread(f'input\mask\mask{index}.jpg')
    
    # Создание объекта аугментации
    transform = alb.Compose([
        alb.HorizontalFlip(p=0.5),
        alb.VerticalFlip(p=0.5),
        alb.RandomRotate90(p=0.5),
        alb.Affine(translate_percent={"x": 0.1, "y": 0.1}, rotate=(-45, 45), scale=(0.8, 1.2), p=1.0),
        alb.RandomScale(scale_limit=0.1, p=0.5), 
        alb.RandomCrop(height=300, width=300, p=0.5), 
        alb.RandomBrightnessContrast(p=0.4),
        alb.Blur(blur_limit=3, p=0.3),
    ])

    # Применение аугментации к изображению
    augmented_i = transform(image=img, mask=mask)
    # augmented_m = transform(image=mask)
    
        # Получение аугментированного изображения
    augmented_image = augmented_i['image']
    augmented_mask = augmented_i['mask']

    # Сохранение аугментированного изображения
    cv2.imwrite(f'augment/img/augmented_image_{index}_{sub_index}.jpg', augmented_image)
    cv2.imwrite(f'augment/mask/augmented_mask_{index}_{sub_index}.jpg', augmented_mask)

In [310]:
count_aug = 10
for i in range(2):
    for num in range(count_aug):
        gen_aug(i, num)

# Машиночитаемый вид

In [311]:
import numpy as np

In [312]:
def get_machine_img(index, sub_index):    
    img = cv2.imread(f'augment/img/augmented_image_{index}_{sub_index}.jpg')
    mask = cv2.imread(f'augment/mask/augmented_mask_{index}_{sub_index}.jpg')
    
    # TODO: аугментировать
    
    # cv2.imshow("img", img)
    # cv2.imshow("mask", mask)
    
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    norm_img = img / 255.0
    norm_mask = (mask / 255.0 > 0).astype(np.float32) # бинаризация
    # cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    return (norm_img, norm_mask)
    

In [313]:
mass_img = []
mass_mask = [] 
for i in range(2):
    for num in range(count_aug):
        img, mask = get_machine_img(i, num)
        print(i, num, img.shape, mask.shape)
        mass_img.append(img), mass_mask.append(mask)

0 0 (300, 300, 3) (300, 300, 3)
0 1 (612, 612, 3) (612, 612, 3)
0 2 (300, 300, 3) (300, 300, 3)
0 3 (300, 300, 3) (300, 300, 3)
0 4 (612, 612, 3) (612, 612, 3)
0 5 (300, 300, 3) (300, 300, 3)
0 6 (300, 300, 3) (300, 300, 3)
0 7 (300, 300, 3) (300, 300, 3)
0 8 (653, 653, 3) (653, 653, 3)
0 9 (614, 614, 3) (614, 614, 3)
1 0 (300, 300, 3) (300, 300, 3)
1 1 (300, 300, 3) (300, 300, 3)
1 2 (300, 300, 3) (300, 300, 3)
1 3 (692, 695, 3) (692, 695, 3)
1 4 (655, 658, 3) (655, 658, 3)
1 5 (300, 300, 3) (300, 300, 3)
1 6 (300, 300, 3) (300, 300, 3)
1 7 (300, 300, 3) (300, 300, 3)
1 8 (666, 662, 3) (666, 662, 3)
1 9 (300, 300, 3) (300, 300, 3)


# Создание модели

In [314]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Полносвязную нейронную сеть желаемой архитектуры.

In [315]:
from tensorflow import keras

In [316]:
mass_img[0].shape, mass_mask[0].shape

((300, 300, 3), (300, 300, 3))

In [317]:
# Создаем модель
model = keras.Sequential()

# Добавляем слои в модель
model.add(keras.layers.Dense(64, activation='relu', input_shape=(mass_img[0].shape)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(mass_mask[0].shape, activation='softmax'))

# Компилируем модель
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучаем модель
model.fit(mass_img, mass_mask, epochs=10, batch_size=32)

c:\PyInterpreters\BigData\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: tuple